# Extracción de características
## Inteligencia Computacional 2021-2, Grupo 8a
Nicolás Canales, Matías Vergara

Este notebook es una reproducción del trabajo realizado por Ignacio Reyes Jainaga, miembro del equipo del broker astronómico ALeRCE. Dicha obra es pública y se encuentra en github: https://github.com/alercebroker/lc_classifier/blob/main/examples/feature_extraction.ipynb


### Instalación de dependencias

In [ ]:
# pyarrow might be needed to read the data
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

Obtaining turbofats from git+https://****@github.com/alercebroker/turbo-fats#egg=turbofats
  Updating ./src/turbofats clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 0e8f0452ce1bcac69ee876c5598a0e1152374400
  Attempting uninstall: turbofats
    Found existing installation: turbofats 2.0.0
    Can't uninstall 'turbofats'. No files were found to uninstall.
  Running setup.py develop for turbofats
Obtaining mhps from git+https://****@github.com/alercebroker/mhps#egg=mhps
  Updating ./src/mhps clone
  Running command git fetch -q --tags
  Running command git reset --hard -q e79e54a1daa9fcda2ad2840afc9adb75112f326f
  Attempting uninstall: mhps
    Found existing installation: mhps 0.0.1
    Can't uninstall 'mhps'. No files were found to uninstall.
  Running setup.py develop for mhps
Obtaining P4J from git+https://****@github.com/alercebroker/P4J#egg=P4J
  Updating ./src/p4j clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 06

### Traer datasets

In [ ]:
!gdown --id 1m4So7GYRKNW7vB9hGDtAvRY8X_u8ymda #data_R.csv

!gdown --id 1_aJr3sT3zXOTB4quDcrD9ZbS7Iwmua3a #data_G.csv

!gdown --id 1KWDAQ_HzeeTqFfDIao3Ix6xB1psnmy9D #present_curves.csv

Downloading...
From: https://drive.google.com/uc?id=1m4So7GYRKNW7vB9hGDtAvRY8X_u8ymda
To: /content/alerts_R.csv
100% 261M/261M [00:01<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_aJr3sT3zXOTB4quDcrD9ZbS7Iwmua3a
To: /content/alerts_G.csv
100% 303M/303M [00:01<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KWDAQ_HzeeTqFfDIao3Ix6xB1psnmy9D
To: /content/present_curves.csv
100% 1.22M/1.22M [00:00<00:00, 18.9MB/s]


### Imports necesarios

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lc_classifier.utils import LightcurveBuilder
from sklearn import preprocessing
import csv

In [ ]:
from lc_classifier.features import MHPSExtractor, PeriodExtractor, GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor, IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor
from lc_classifier.features import FeatureExtractorComposer

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
alerts_G = pd.read_csv("/content/alerts_G.csv",
                       index_col=0)
alerts_R = pd.read_csv("/content/alerts_R.csv",
                       index_col=0)
present_curves = pd.read_csv("/content/present_curves.csv",
                             index_col=None)

### Estructurando la data
Los dataframes disponibles en `alerts_G.csv` y `alerts_R.csv` corresponden a observaciones de estrellas periódicas en las bandas G y R respectivamente. Sus columnas son oid, magnitud, error y tiempo, que no coincide con aquellas requeridas por la librería (oid, time, magnitude, error, band). En esta sección se reestructurará la data para que tome la forma necesaria.

Primero vamos por las alertas en G

In [ ]:
alerts_G.head()

,magpsf_corr,mjd,sigmapsf_corr,sigmapsf_corr_ext
oid,,,,
ZTF19aadtxkl,16.913750,58634.209421,0.012275,0.021475
ZTF18abcqlby,13.206103,58592.459630,100.000000,0.009922
ZTF19aaxlzmv,16.038790,58677.317847,0.028514,0.028599
ZTF18abcqhso,12.774134,58733.330463,100.000000,0.006951
ZTF18abcqhtc,12.619893,58363.175486,100.000000,0.009305


In [ ]:
g = []
for i in range(0, len(alerts_G.index)):
  g.append('g')
alerts_G['band'] = g
alerts_G.head()

,magpsf_corr,mjd,sigmapsf_corr,sigmapsf_corr_ext,band
oid,,,,,
ZTF19aadtxkl,16.913750,58634.209421,0.012275,0.021475,g
ZTF18abcqlby,13.206103,58592.459630,100.000000,0.009922,g
ZTF19aaxlzmv,16.038790,58677.317847,0.028514,0.028599,g
ZTF18abcqhso,12.774134,58733.330463,100.000000,0.006951,g
ZTF18abcqhtc,12.619893,58363.175486,100.000000,0.009305,g


In [ ]:
alerts_G.rename(columns = {'mjd': 'time',
                           'magpsf_corr': 'magnitude',
                           'sigmapsf_corr': 'error',
                           'sigmapsf_corr_ext': 'error_ext'}, inplace=True)
alerts_G.head()

,magnitude,time,error,error_ext,band
oid,,,,,
ZTF19aadtxkl,16.913750,58634.209421,0.012275,0.021475,g
ZTF18abcqlby,13.206103,58592.459630,100.000000,0.009922,g
ZTF19aaxlzmv,16.038790,58677.317847,0.028514,0.028599,g
ZTF18abcqhso,12.774134,58733.330463,100.000000,0.006951,g
ZTF18abcqhtc,12.619893,58363.175486,100.000000,0.009305,g


Ahora procesamos las alertas en R

In [ ]:
alerts_R.head()

,magpsf_corr,mjd,sigmapsf_corr,sigmapsf_corr_ext
oid,,,,
ZTF19aadtwtk,14.796465,58581.361204,100.000000,0.007631
ZTF19aadtwtl,16.360971,58877.501366,0.019957,0.022477
ZTF19aadtzqg,16.385742,58854.459456,0.026998,0.027591
ZTF18abcqled,13.972938,58376.354271,100.000000,0.016313
ZTF18abcqled,14.341584,58776.127755,100.000000,0.023839


In [ ]:
r = []
for i in range(0, len(alerts_R.index)):
  r.append('r')
alerts_R['band'] = r

In [ ]:
alerts_R.rename(columns = {'mjd': 'time',
                           'magpsf_corr': 'magnitude',
                           'sigmapsf_corr': 'error',
                           'sigmapsf_corr_ext': 'error_ext'}, inplace=True)
alerts_R.head()

,magnitude,time,error,error_ext,band
oid,,,,,
ZTF19aadtwtk,14.796465,58581.361204,100.000000,0.007631,r
ZTF19aadtwtl,16.360971,58877.501366,0.019957,0.022477,r
ZTF19aadtzqg,16.385742,58854.459456,0.026998,0.027591,r
ZTF18abcqled,13.972938,58376.354271,100.000000,0.016313,r
ZTF18abcqled,14.341584,58776.127755,100.000000,0.023839,r


Juntamos ambos dataframe en uno, para pasarselo al extractor

In [ ]:
light_curves = pd.concat([alerts_G, alerts_R])
print(len(light_curves.index))

#limpieza de filas con valores nan
light_curves.dropna(inplace=True)
print(len(light_curves.index))
df = light_curves

# limpieza de valores negativos o 0 (no hay)
df = df.drop(df[df.time <= 0].index)
df = df.drop(df[df.magnitude <= 0].index)
df = df.drop(df[df.error <= 0].index)

9003067
8935910


In [ ]:
# estandarizar fechas para evitar overflows
min_fecha = df['time'].min()
df['time'] = df['time'] - min_fecha
df

,magnitude,time,error,error_ext,band
oid,,,,,
ZTF19aadtxkl,16.913750,392.061482,0.012275,0.021475,g
ZTF18abcqlby,13.206103,350.311690,100.000000,0.009922,g
ZTF19aaxlzmv,16.038790,435.169907,0.028514,0.028599,g
ZTF18abcqhso,12.774134,491.182523,100.000000,0.006951,g
ZTF18abcqhtc,12.619893,121.027546,100.000000,0.009305,g
...,...,...,...,...,...
ZTF17aabszub,15.957872,745.014896,100.000000,0.008658,r
ZTF17aabszuc,13.908604,284.041817,100.000000,0.006144,r
ZTF17aabszxg,15.128976,613.179896,0.002739,0.010037,r


Dropearemos la columna error pues tiene muchos valores 100 (alrededor del 50% de los datos), y mantendremos en su lugar error_ext (recomendación del auxiliar)

In [ ]:
df = df.drop('error',1)

In [ ]:
df.rename(columns = {'error_ext':'error'}, inplace=True)
light_curves = df
light_curves

,magnitude,time,error,band
oid,,,,
ZTF19aadtxkl,16.913750,392.061482,0.021475,g
ZTF18abcqlby,13.206103,350.311690,0.009922,g
ZTF19aaxlzmv,16.038790,435.169907,0.028599,g
ZTF18abcqhso,12.774134,491.182523,0.006951,g
ZTF18abcqhtc,12.619893,121.027546,0.009305,g
...,...,...,...,...
ZTF17aabszub,15.957872,745.014896,0.008658,r
ZTF17aabszuc,13.908604,284.041817,0.006144,r
ZTF17aabszxg,15.128976,613.179896,0.010037,r


In [ ]:
light_curves.index.unique()

Index(['ZTF19aadtxkl', 'ZTF18abcqlby', 'ZTF19aaxlzmv', 'ZTF18abcqhso',
       'ZTF18abcqhtc', 'ZTF18abcqkag', 'ZTF18abcqkai', 'ZTF18abcqkak',
       'ZTF18abcqkal', 'ZTF18abcqkas',
       ...
       'ZTF18abluuqy', 'ZTF18abkitlx', 'ZTF18aabqxsp', 'ZTF18acvrxxj',
       'ZTF18abrnghw', 'ZTF19aawqhnv', 'ZTF18abtoutu', 'ZTF18abaynhe',
       'ZTF18abpatzy', 'ZTF18abtghnx'],
      dtype='object', name='oid', length=87015)

### Computando características sobre el dataset

Para computar las características le damos el dataframe con observaciones al método `compute_features` de nuestro extractor de características. Las características se mostrarán como columnas y los objetos como filas.

In [ ]:
bands = ['g', 'r']
feature_extractor = FeatureExtractorComposer(
    [ # probar comentando alguno de estos 
        PeriodExtractor(bands),
        GPDRWExtractor(bands),
        FoldedKimExtractor(bands),
        HarmonicsExtractor(bands),
        IQRExtractor(bands),
        PowerRateExtractor(bands),
        TurboFatsFeatureExtractor(bands)
    ]
)

In [ ]:
init_bold = "\033[1m"
end_bold = "\033[0m"
output_features = open("features_lc.csv", 'w')
out_writer = csv.writer(output_features, delimiter=",")
header = False
for oid in light_curves.index.unique():
  try:
    features = feature_extractor.compute_features(light_curves.loc[oid])
    print(init_bold + "Exito con oid {}".format(oid) + end_bold)
    if not header:
      out_writer.writerow(features.columns)
      header = True
    out_writer.writerow(features.loc[oid].values)
  except:
    print(init_bold + "Error con oid {}".format(oid) + end_bold)

Exito con oid ZTF19aadtxkl


 /content/src/turbofats/turbofats/FeatureFunctionLib.py:687: RuntimeWarning:divide by zero encountered in true_divide
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:693: RuntimeWarning:invalid value encountered in double_scalars
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:437: RuntimeWarning:divide by zero encountered in true_divide


Exito con oid ZTF18abcqlby
Exito con oid ZTF19aaxlzmv
Exito con oid ZTF18abcqhso


 /content/src/turbofats/turbofats/FeatureFunctionLib.py:690: RuntimeWarning:invalid value encountered in multiply
 /content/src/turbofats/turbofats/FeatureFunctionLib.py:437: RuntimeWarning:invalid value encountered in true_divide


Exito con oid ZTF18abcqhtc
Exito con oid ZTF18abcqkag
Exito con oid ZTF18abcqkai
Exito con oid ZTF18abcqkak
Exito con oid ZTF18abcqkal
Exito con oid ZTF18abcqkas
Exito con oid ZTF18abcqkfw
Exito con oid ZTF18abcqkvy
Exito con oid ZTF18aaxdoee
Exito con oid ZTF18aaxdoeg
Exito con oid ZTF17aabukqy
Exito con oid ZTF17aabukrc
Exito con oid ZTF17aabukre
Exito con oid ZTF17aabukru
Exito con oid ZTF17aabuktp
Exito con oid ZTF17aabukuh
Exito con oid ZTF17aabukyt
Exito con oid ZTF17aabukze
Exito con oid ZTF17aabulac
Exito con oid ZTF17aabulbj
Exito con oid ZTF17aabuldp
Exito con oid ZTF18abfipnd
Exito con oid ZTF18abehiid
Exito con oid ZTF18abehirw
Exito con oid ZTF18abehjcp
Exito con oid ZTF18abehjiw
Exito con oid ZTF18abehjzr
Exito con oid ZTF18abfhxvk
Exito con oid ZTF17aabuknv
Exito con oid ZTF17aabukoi
Exito con oid ZTF17aabukoo
Exito con oid ZTF17aabukow
Exito con oid ZTF18abfison
Exito con oid ZTF18abfisxn
Exito con oid ZTF18abfiton
Exito con oid ZTF18abfitvt
Exito con oid ZTF18abfixlg
E

...

Exito con oid ZTF18acrvajr
Exito con oid ZTF18acrvall
Exito con oid ZTF18acrvama
Exito con oid ZTF18acrvamj
Exito con oid ZTF18acrvano
Exito con oid ZTF18acrvapx
Exito con oid ZTF18acrvasc
Exito con oid ZTF18abnbgqr
Exito con oid ZTF18abnbiie
Exito con oid ZTF18abnbiwm
Exito con oid ZTF18abnbiwp
Exito con oid ZTF18abnbiyp
Exito con oid ZTF18abnbjtp
Exito con oid ZTF18abmjgti
Exito con oid ZTF18abmjgwd
Exito con oid ZTF18abmjhbh
Exito con oid ZTF18aaafmnc
Exito con oid ZTF18aaafnop
Exito con oid ZTF18aaafruj
Exito con oid ZTF18aaafrwc
Exito con oid ZTF18aaafsur
Exito con oid ZTF18aaaftdt
Exito con oid ZTF18aaafueb
Exito con oid ZTF18aaafugc
Exito con oid ZTF18aaafuti
Exito con oid ZTF18aaafvfg
Exito con oid ZTF18aaafvip
Exito con oid ZTF18aaafwgc
Exito con oid ZTF18aaafyug
Exito con oid ZTF18aaafyvv
Exito con oid ZTF18aaafyxz
Exito con oid ZTF18aaafyyy
Exito con oid ZTF18aaafzfk
Exito con oid ZTF18aaagagg
Exito con oid ZTF18aaagbkq
Exito con oid ZTF18aaagbry
Exito con oid ZTF19abdjzxa
E

 /content/src/turbofats/turbofats/FeatureSpace.py:57: RankWarning:Polyfit may be poorly conditioned


Exito con oid ZTF18abnuekd
Exito con oid ZTF18abnuekv
Exito con oid ZTF18abnuele
Exito con oid ZTF18abnuelq
Exito con oid ZTF18abnuene
Exito con oid ZTF18abnueos
Exito con oid ZTF18abnuepv
Exito con oid ZTF18abnueqk
Exito con oid ZTF18abnuevg
Exito con oid ZTF18abnuexf
Exito con oid ZTF18abnuexx
Exito con oid ZTF18abnueyc
Exito con oid ZTF18abnuezm
Exito con oid ZTF18abnufdm
Exito con oid ZTF18abnufgu
Exito con oid ZTF18abnufht
Exito con oid ZTF18abnufrb
Exito con oid ZTF18abnufvf
Exito con oid ZTF18abnufxg
Exito con oid ZTF18abnufyh
Exito con oid ZTF18abgkhbv
Exito con oid ZTF18abgkjcc
Exito con oid ZTF18abgkjkb
Exito con oid ZTF18aaxddkg
Exito con oid ZTF18aaxddmv
Exito con oid ZTF18aaxddqu
Exito con oid ZTF18aaxddvz
Exito con oid ZTF18abdkcyq
Exito con oid ZTF18abdkdai
Exito con oid ZTF18abdkdbd
Exito con oid ZTF18abdkdjr
Exito con oid ZTF18abdkdth
Exito con oid ZTF18adacqkj
Exito con oid ZTF18adactox
Exito con oid ZTF18abscxfj
Exito con oid ZTF18absdadd
Exito con oid ZTF18absdadh
E

 /usr/local/lib/python3.7/dist-packages/celerite2/core.py:427: RuntimeWarning:invalid value encountered in double_scalars


Exito con oid ZTF18aazazia
Exito con oid ZTF18aazcyqp
Exito con oid ZTF18aazdbkj
Exito con oid ZTF18aazdbxy
Exito con oid ZTF18aazdcdy
Exito con oid ZTF18aazdcxx
Exito con oid ZTF18aazdete
Exito con oid ZTF18aazdetk
Exito con oid ZTF18aazdfyr
Exito con oid ZTF18aazdgao
Exito con oid ZTF18aazdgki
Exito con oid ZTF18aazdjyv
Exito con oid ZTF18aazdkdu
Exito con oid ZTF18aazdpdl
Exito con oid ZTF18aazdtkg
Exito con oid ZTF18aazehma
Exito con oid ZTF18abmlxtk
Exito con oid ZTF18abmlxuz
Exito con oid ZTF18abmlxvb
Exito con oid ZTF18abmlygd
Exito con oid ZTF18acmftft
Exito con oid ZTF18acmfvbz
Exito con oid ZTF18acmfxls
Exito con oid ZTF18acmfxmd
Exito con oid ZTF18aaajkbk
Exito con oid ZTF18aaajkqd
Exito con oid ZTF18aaajkrz
Exito con oid ZTF18aaajkxz
Exito con oid ZTF18aaajlst
Exito con oid ZTF18aaajmty
Exito con oid ZTF18aaajmtz
Exito con oid ZTF18aaajona
Exito con oid ZTF18aaajpek
Exito con oid ZTF18aaajphl
Exito con oid ZTF18aaajqaf
Exito con oid ZTF18aaajqcm
Exito con oid ZTF18aaajqct
E

In [ ]:
period_feature_names = [f for f in features.columns if 'period' in f.lower()]